In [4]:
import numpy as np

In [5]:
from numpy import random
#   BidF = []
#   for rnd in random.uniform(0,1, size = [1,10])[0]:
#           f = lambda currentBid, bidHist, rnd2=rnd: rnd2
#           BidF.append( f )

In [6]:
np.random.uniform(0,1, size = [1,10])[0]

array([ 0.01119695,  0.73272951,  0.07360154,  0.26588898,  0.43222601,
        0.05382469,  0.91001334,  0.85000715,  0.17764414,  0.13357577])

In [7]:
def evaluator(price_format, auction_format, bidder_info, bid_history=[], curr_bid=0., bid_inc=0.):
    """
    price_format (int): e.g. 1 for 1st price and 2 for 2nd price
    auction_format (str): 'OO' for open outcry and 'SB' for sealed bid
    bid_history (ordered list of dict): 
        For SB: e.g. [{"Round": 1, "Value": None ,  "Bids": [1.25, 0, .85] }]
        For OO: e.g. [{ "Round" : 1, "Value": .25,  "Bids": [1, 1, 1]},
                      { "Round" : 2, "Value": .5,  "Bids": [1, 1, 0]},
                      { "Round" : 3, "Value": .75,  "Bids": [1, 0, 0]}]
    bidder_info (list of dict):
        e.g. [{"BidderName" : "Bidder 1", "BidFunction" : B1},
              {"BidderName" : "Bidder 2", "BidFunction" : B2},
              {"BidderName" : "Bidder 3", "BidFunction" : B3},
              {"BidderName" : "Bidder 4", "BidFunction" : B4}]
    curr_bid (float): current value to bid (only for OO)
    bid_inc (float): bid increase each round (only for OO)
    returns (winning_bidder_name (str), winning_bid_value (float))
    """
    if auction_format == 'SB':
        bids = []
        bidders = []
        max_bid = 0
        for bidder in bidder_info:
            bids.append(bidder['BidFunction'](curr_bid, bid_history))
            bidders.append(bidder['BidderName'])
        return np.random.choice(np.array(bidders)[np.array(bids)==max(bids)]), sorted(bids, reverse=True)[price_format - 1]
    
    if auction_format == 'OO':
        n_round = len(bid_history) + 1
        bids = np.array([1 if bidder['BidFunction'](curr_bid, bid_history) >= curr_bid else 0 for bidder in bidder_info])
        bid_history.append({"Round": n_round, "Value": curr_bid, "Bids": bids})
        if max(bids) == 0: # no one bids in the current round
            if n_round - 1 < price_format: # not enough rounds with bids to have a winner given the price format
                return None, None
            bids = np.array(bid_history[-price_format-1]['Bids']) # else trace back the history to find the winning bid
            return bidder_info[np.random.choice(np.arange(len(bidder_info))[bids==1])]['BidderName'], bid_history[-price_format-1]['Value']
        if sum(bids==1) == 1: # there is only 1 bidder in the current round
            return bidder_info[np.argmax(bids)]['BidderName'], curr_bid
        # else there are more than one bidder. Go to the next round with increased value
        return evaluator(price_format, auction_format, bidder_info, bid_history, curr_bid + bid_inc, bid_inc)
    print 'Invalid auction format'
    return None

### Case (a)

In [8]:
BF = []
for b in np.arange(0, 1, 0.1):
    BF.append(lambda curr_bid, bid_history, val=b: val)
BI = [{'BidderName': 'Bidder %s' %str(i+1), 'BidFunction': BF[i]} for i in range(10)]

In [9]:
winner, revenue = evaluator(2, 'SB', BI)
print 'The winner is %s, with revenue of %.2f' %(winner, revenue)

The winner is Bidder 10, with revenue of 0.80


### Case (b)

In [10]:
BF = []
for b in np.arange(0, 1, 0.1):
    BF.append(lambda curr_bid, bid_history, val=b: 9. * val / 10.)
BI = [{'BidderName': 'Bidder %s' %str(i+1), 'BidFunction': BF[i]} for i in range(10)]

In [11]:
winner, revenue = evaluator(1, 'SB', BI)
print 'The winner is %s, with revenue of %.2f' %(winner, revenue)

The winner is Bidder 10, with revenue of 0.81


### Case (c)

In [16]:
BF = []
for b in np.arange(0, 1, 0.1):
    BF.append(lambda curr_bid, bid_history, val=b: val)
BI = [{'BidderName': 'Bidder %s' %str(i+1), 'BidFunction': BF[i]} for i in range(10)]

In [17]:
BH = []
winner, revenue = evaluator(2, 'OO', BI, BH, 0, 0.05)
print 'The winner is %s, with revenue of %.2f\n%d rounds of auction was needed' %(winner, revenue, len(BH))

The winner is Bidder 10, with revenue of 0.80
17 rounds of auction was needed
